<a href="https://colab.research.google.com/github/L-ansari/NLP/blob/master/attentionLstmUpdated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
print(torch.__version__)


1.7.0+cu101


In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
df_train = pd.read_csv(Path('/content/drive/My Drive/DSP/data/bachelor/train.csv'))


In [ ]:
df_train.head()

,label,text
0,depression_ctrl,Game over RT : Life is so unpredictable. Be gr...
1,depression,Happy ! Guys: please check your balls monthly ...
2,ptsd_ctrl,You gotta enjoy the view RT : i want a boyfrie...
3,ptsd_ctrl,I love that I'm so chill about everything RT :...
4,depression_ctrl,Photo: lkfrannyfrano: hotshxxt: migzfromatx: s...


In [ ]:
print(df_train['label'][1])

depression


In [ ]:
for i in range (0, (df_train.shape[0])):
    if (df_train['label'][i]=='depression' or  df_train['label'][i]=='depression_ctrl') :
        
        df_train['label'][i]= 'positive' 
    else:
        #False
        df_train['label'][i]= 'negative' 
    
df_train.head()


,label,text
0,positive,Game over RT : Life is so unpredictable. Be gr...
1,positive,Happy ! Guys: please check your balls monthly ...
2,negative,You gotta enjoy the view RT : i want a boyfrie...
3,negative,I love that I'm so chill about everything RT :...
4,positive,Photo: lkfrannyfrano: hotshxxt: migzfromatx: s...


In [ ]:
df_test = pd.read_csv(Path('/content/drive/My Drive/DSP/data/bachelor/test.csv'))


In [ ]:
for i in range (0, (df_test.shape[0])):
    if (df_test['label'][i]=='depression' or  df_test['label'][i]=='depression_ctrl') :
        
        df_test['label'][i]= 'positive' 
    else:
        #False
        df_test['label'][i]= 'negative' 
    
df_test.head()


,label,text
0,negative,: the is happening... the balance of life...te...
1,positive,: Going with . Look for my lipstick in early t...
2,negative,Fox News: Second wave of health plan cancellat...
3,negative,no. The needles are just different for shading...
4,negative,On holiday... : Where's your favorite place to...


In [ ]:
#export train only needed columns
df = pd.DataFrame(df_train, columns = ['text', 'label']) 
df.to_csv('train', index=False)


In [ ]:
#export test only needed columns
dff = pd.DataFrame(df_test, columns = ['text', 'label']) 
dff.to_csv('test', index=False)


In [ ]:
import torch
import torchtext
from torchtext import data
from torchtext.vocab import Vectors, GloVe
import torch.utils.data as data_utils
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
tokenize = lambda x: x.split()
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
LABEL = data.LabelField(sequential=False)


In [ ]:
fields = [('text', TEXT), ('label', LABEL)]

In [ ]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [ ]:
train_data, test_data = data.TabularDataset.splits(
    path='/content/',
    train='train',
    test='test',
    format='csv',
    fields=fields,
    skip_header=True
)

In [ ]:
print(train_data.shape)

<generator object Dataset.__getattr__ at 0x7f95f231bfc0>


In [ ]:
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))


In [ ]:
LABEL.build_vocab(train_data)

In [ ]:
word_embeddings = TEXT.vocab.vectors

In [ ]:
word_embeddings = TEXT.vocab.vectors
print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
print ("Label Length: " + str(len(LABEL.vocab)))

Length of Text Vocabulary: 157915
Vector size of Text Vocabulary:  torch.Size([157915, 300])
Label Length: 2


In [ ]:

train_data, valid_data = train_data.split(split_ratio=0.6) # Further splitting of training_data to create new training_data & validation_data
train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

vocab_size = len(TEXT.vocab)

In [ ]:
print(torchtext.__version__)

0.3.1


In [ ]:
"""import os
import time
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
"""

'import os\nimport time\nimport torch\nimport torch.nn.functional as F\nfrom torch.autograd import Variable\nimport torch.optim as optim\nimport numpy as np\n'

In [ ]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    #Remove any epoch loss or accuracy from the previous run
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()     
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()        #Set the model to training mode
    for idx, batch in enumerate(train_iter): #for each bacth in the iterator
        text = batch.text[0]
        target = batch.label.long()
        #target.requires_grad=True
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] is not 32):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()         #1. Clean the old gradients
        prediction = model(text)  #2. Forward pass
        loss = loss_fn(prediction, target)    #3. Calculate the loss
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()         #4. Perform gradient decent differentiation backwards 
        clip_gradient(model, 1e-1)
        optim.step()         # 5. Gradient step: updates the weights for the model
        steps += 1 
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()  # Aggregate the epoch loss and accuracy within a given bacth
        total_epoch_acc += acc.item()  #The item method returns a scaler
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter) #return average epoch loss & accuracy


In [ ]:
def eval_model(model, val_iter):
    #Remove any epoch loss or accuracy from the previous run
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()               #Set the model to evaluation mode-->disables batch normalization & dropouts
    with torch.no_grad():       #   We don't update the weights & gradients are not reqiured-->prevent gradient calculations
        for idx, batch in enumerate(val_iter): #for each batch in the iterator
            text = batch.text[0]
            if (text.size()[0] is not 32):
                continue
            target = batch.label.long()
            #target = torch.autograd.target.long()
            #target.requires_grad = True
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction = model(text)
            loss = loss_fn(prediction, target)
            #From all the prediction how many were right
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item() # Aggregate the epoch loss and accuracy within a given bacth
            total_epoch_acc += acc.item()  #The item method returns a scaler

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter) #return average epoch loss & accuracy

In [ ]:
class LSTMClassifier(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(LSTMClassifier, self).__init__()
		
		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		"""
		
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)# Initializing the look-up table.
		self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False) # Assigning the look-up table to the pre-trained GloVe word embedding.
		self.lstm = nn.LSTM(embedding_length, hidden_size)
		self.label = nn.Linear(hidden_size, output_size)
		
	def forward(self, input_sentence, batch_size=None):
	
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for positive & negative class which receives its input as the final_hidden_state of the LSTM
		final_output.shape = (batch_size, output_size)
		
		"""
		
		''' Here we will map all the indexes present in the input sequence to the corresponding word vector using our pre-trained word_embedddins.'''
		input = self.word_embeddings(input_sentence) # embedded input of shape = (batch_size, num_sequences,  embedding_length)
		input = input.permute(1, 0, 2) # input.size() = (num_sequences, batch_size, embedding_length)
		if batch_size is None:
			h_0 = torch.zeros(1, self.batch_size, self.hidden_size).cuda() # Initial hidden state of the LSTM
			c_0 = torch.zeros(1, self.batch_size, self.hidden_size).cuda() # Initial cell state of the LSTM
		else:
			h_0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
			c_0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
   
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
		final_output = self.label(final_hidden_state[-1]) # final_hidden_state.size() = (1, batch_size, hidden_size) & final_output.size() = (batch_size, output_size)
		
		return final_output


In [ ]:
import torch
import torch.nn as nn
#from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np

class AttentionModel(torch.nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(AttentionModel, self).__init__()
		
		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		--------
		
		"""
		
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length

		#Layers within model:  Embedding - LSTM  - Linear (Fully connected).
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
		self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
		self.lstm = nn.LSTM(embedding_length, hidden_size)
		self.label = nn.Linear(hidden_size, output_size)
		#self.attn_fc_layer = nn.Linear()
		
	def attention_net(self, lstm_output, final_state):

		""" 
		Attention is used to compute soft alignment score corresponding
		between each of the hidden_state and the last hidden_state of the LSTM. 
		
		Arguments
		---------
		
		lstm_output : Final output of the LSTM which contains hidden layer outputs for each sequence.
		final_state : Final time-step hidden state (h_n) of the LSTM
		
		---------
		
		Returns : It performs attention mechanism by first computing weights for each of the sequence present in lstm_output and and then finally computing the
				  new hidden state.
				  
		Tensor Size :
					hidden.size() = (batch_size, hidden_size)
					attn_weights.size() = (batch_size, num_seq)
					soft_attn_weights.size() = (batch_size, num_seq)
					new_hidden_state.size() = (batch_size, hidden_size)
					  
		"""
		
		hidden = final_state.squeeze(0)
		attn_weights = torch.bmm(lstm_output, hidden.unsqueeze(2)).squeeze(2)
		soft_attn_weights = F.softmax(attn_weights, 1)
		new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
		
		return new_hidden_state
	
	def forward(self, input_sentences, batch_size=None):
	
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class which receives its input as the new_hidden_state which is basically the output of the Attention network.
		final_output.shape = (batch_size, output_size)
		
		"""
		#First: Embedding
		input = self.word_embeddings(input_sentences)
		input = input.permute(1, 0, 2)
		if batch_size is None:
			h_0 = torch.zeros(1, self.batch_size, self.hidden_size).cuda()
			c_0 = torch.zeros(1, self.batch_size, self.hidden_size).cuda()
		else:
			h_0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
			c_0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
		
    #Second: LSTM - response consists of output state and a tupple of (hidden state, cell state)
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0)) # final_hidden_state.size() = (1, batch_size, hidden_size) 
		output = output.permute(1, 0, 2) # output.size() = (batch_size, num_seq, hidden_size)
		#Third: Attention
		attn_output = self.attention_net(output, final_hidden_state)
    #Fourth linear fully connected
		logits = self.label(attn_output)
		
		return logits

In [ ]:
learning_rate = 2e-5
batch_size = 32
output_size = 2
hidden_size = 256
embedding_length = 300

#model = LSTMClassifier(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
model = AttentionModel(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)


loss_fn = F.cross_entropy

In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
for epoch in range(20):

    #Forward pass- Calculate loss- Perform GD- Take gradient step- Zero gradients
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

''' Let us now predict the sentiment on a single sentence just for the testing purpose. '''
test_sen1 = "Life is good."
test_sen2 = "The end is close."

test_sen1 = TEXT.preprocess(test_sen1)
test_sen1 = [[TEXT.vocab.stoi[x] for x in test_sen1]]

test_sen2 = TEXT.preprocess(test_sen2)
test_sen2 = [[TEXT.vocab.stoi[x] for x in test_sen2]]

test_sen = np.asarray(test_sen1)
test_sen = torch.LongTensor(test_sen)
test_tensor = test_sen
test_tensor = test_tensor.cuda()
model.eval()


output = model(test_tensor, 1)
#Feed the output to softmax to get the prediction
out = F.softmax(output, 1)

if (torch.argmax(out[0]) == 1): 
    print ("Sentiment: Positive")
else:
    print ("Sentiment: Negative")


Epoch: 01, Train Loss: 0.349, Train Acc: 25.00%, Val. Loss: 0.335421, Val. Acc: 32.81%
Epoch: 02, Train Loss: 0.328, Train Acc: 35.94%, Val. Loss: 0.340152, Val. Acc: 31.25%
Epoch: 03, Train Loss: 0.301, Train Acc: 42.19%, Val. Loss: 0.330744, Val. Acc: 35.94%
Epoch: 04, Train Loss: 0.280, Train Acc: 46.88%, Val. Loss: 0.322149, Val. Acc: 34.38%
Epoch: 05, Train Loss: 0.265, Train Acc: 42.19%, Val. Loss: 0.324759, Val. Acc: 34.38%
Epoch: 06, Train Loss: 0.237, Train Acc: 40.62%, Val. Loss: 0.334818, Val. Acc: 29.69%
Epoch: 07, Train Loss: 0.216, Train Acc: 45.31%, Val. Loss: 0.333135, Val. Acc: 32.81%
Epoch: 08, Train Loss: 0.172, Train Acc: 48.44%, Val. Loss: 0.348581, Val. Acc: 29.69%
Epoch: 09, Train Loss: 0.123, Train Acc: 50.00%, Val. Loss: 0.381075, Val. Acc: 26.56%
Epoch: 10, Train Loss: 0.100, Train Acc: 48.44%, Val. Loss: 0.439142, Val. Acc: 28.12%
Epoch: 11, Train Loss: 0.075, Train Acc: 50.00%, Val. Loss: 0.479116, Val. Acc: 26.56%
Epoch: 12, Train Loss: 0.057, Train Acc: 50